In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
import sklearn.model_selection as ms
import numpy as np
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import *
from sklearn.model_selection import GroupShuffleSplit
from sklearn.metrics import accuracy_score

In [2]:
df = pd.read_parquet('teDataV2.parquet')

In [3]:
df = df.dropna(subset=['fantasy_points_ppr_future'])

In [4]:
import plotly.express as px
fig = px.ecdf(df, x="fantasy_points_ppr_per_game")
fig.show()

In [5]:
tier = []
thresholds = [14, 12, 10, 8, df['fantasy_points_ppr_per_game_future'].min()]
for row in df.itertuples():
    for i in range(len(thresholds)):
        if row.fantasy_points_ppr_per_game_future >= thresholds[i]:
            tier.append(i)
            break

In [6]:
df['tier_future'] = tier

In [7]:
temp = df[['season',
    'fantasy_points_ppr_per_game', 
    'fantasy_points_ppr',
    'fantasy_rank', 
    'fantasy_points_ppr_per_game_future', 
    'fantasy_points_ppr_future',
    'fantasy_rank_future',
    'tier_future']]

In [8]:
colsToDrop = ['player_id', 'season',
              'fantasy_points_ppr_future', 
              'fantasy_points_ppr_per_game_future',
              'fantasy_rank_future',
              'tier_future']

In [9]:
# separate featues and labels
features = df.drop(colsToDrop, axis=1)
labels = df['tier_future']
# train-test split
x_train, x_val, y_train, y_val = train_test_split(features, labels, train_size=0.85)

In [10]:

# fit model no training data
model = xgb.XGBClassifier()
model.fit(x_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, objective='multi:softprob',
              predictor='auto', random_state=0, reg_alpha=0, ...)

In [11]:
print(model)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, objective='multi:softprob',
              predictor='auto', random_state=0, reg_alpha=0, ...)


In [12]:

# make predictions for test data
y_pred = model.predict(x_val)
predictions = [round(value) for value in y_pred]

In [13]:

# evaluate predictions
accuracy = accuracy_score(y_val, predictions)

In [14]:
accuracy

0.9494949494949495

In [15]:
px.histogram(predictions)

In [16]:
px.histogram(y_val)